In [1]:
# Install required packages based on your python version
# Please install beautifulsoup4 if you have not done it
!!pip install beautifulsoup4

import numpy as np
import pandas as pd
import requests
from datetime import datetime
import yfinance as yf
from bs4 import BeautifulSoup
import json

In [2]:
# Roce calculation
'''
ROCE = (Operating Income / share holder equity) * 100
'''

def calc_roce(operating_income, share_holder_equity):
    return (operating_income / share_holder_equity) * 100

In [3]:
# price earning ratio
'''
PRICE_EARN_RATIO = (Stock Price / Earning Per Ratio) * 100 
'''

def calc_price_earn_ratio(stock_price, earning_per_ratio):
    return (stock_price/earning_per_ratio) * 100

In [4]:
# Earning Per Share
'''
Earning Per Share = (Net Income / Number of Ordinary Share) 
'''

def earning_per_share(net_income, number_of_ordinary_share):
    return (net_income / number_of_ordinary_share) 

In [5]:
# Liquidity Ratio
'''
Liquidity Ratio = (Current Assets / Current Liabilities)
'''
def calc_liquidity_ratio(current_assets,current_liabilities):
    return (current_assets/current_liabilities)

In [7]:
def asses_good_stocks_yahoo(stock, headers):
    
    IS  = requests.get(f"https://stockanalysis.com/stocks/{stock}/financials/", headers=headers)
    soup = BeautifulSoup(IS.content, 'html.parser')
    data = json.loads(soup.find('script', type='application/json').string)
    
    earning_per_share = float(data['data']['data']['eps'][1]) # get eps from income-statement
    operating_income = float(data['data']['data']['opinc'][1]) # get operating income from income-statment
    #dt = data['props']['pageProps']['data']['datekey'][1]
    dt = data['data']['data']['datekey'][1]
    
    BS = requests.get(f"https://stockanalysis.com/stocks/{stock}/financials/balance-sheet/", headers=headers)
    soup = BeautifulSoup(BS.content, 'html.parser')
    data = json.loads(soup.find('script', type='application/json').string) 
    
    
    share_holder_equity = float(data['data']['data']['equity'][1]) # get share holder equity from balance sheet
    total_current_assets = float(data['data']['data']['assetsc'][1]) # total number of assets from balance sheet
    total_current_liabilities = float(data['data']['data']['liabilitiesc'][1]) # liabilities from income stmt
    
    # get the particular date for calculating price earning ratio
    dt_obj = datetime.date(datetime.strptime(dt, '%Y-%m-%d'))
    data = yf.download(stock, 
                      start=dt_obj, 
                      end=dt_obj,
                      progress=False)
    stock_price = data['Adj Close'][0]
    
    ROCE = calc_roce(operating_income, share_holder_equity)
    price_earning_ratio = calc_price_earn_ratio(stock_price, earning_per_share)
    liquidity_ratio = calc_liquidity_ratio(total_current_assets, total_current_liabilities)
    #print('ROCE: ', + ROCE) 
    #print('Price earning ratio', + price_earning_ratio)
    #print('EPS: ', + earning_per_share)
    #print('Liquidity Ratio: ', +liquidity_ratio)
    return ROCE, price_earning_ratio, earning_per_share, liquidity_ratio
    
    
headers= {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Cache-Control': 'max-age=0'
}

stocks = ['AAPL', 'GOOG', 'MSFT', 'AMZN', 'TSLA','TGT', 'UNH', 'V', 'DIS', 'KO']
dict_stock = {}
for stock in stocks:
    #asses_good_stocks_yahoo(stock, headers)
    dict_stock[stock] = asses_good_stocks_yahoo(stock, headers)

dict_stock = pd.DataFrame.from_dict(dict_stock, orient='index', columns = ['ROCE', 'PRICE_EARNING_RAIO', 
                                                                          'EPS', 'Liquidity_RATIO'])
print(dict_stock)

AttributeError: 'NoneType' object has no attribute 'string'